In [1]:
movies = [
    {
        "id": "movie1",
        "title": "7번방의 선물",
        "year": 2013,
        "genre": "드라마",
        "description": "억울한 누명을 쓰고 교도소에 수감된 아빠와 그의 어린 딸의 감동적인 스토리"
    },
    {
        "id": "movie2",
        "title": "미나리",
        "year": 2020,
        "genre": "드라마",
        "description": "한국계 미국인 가족의 따뜻하고 감성적인 성장 이야기"
    },
    {
        "id": "movie3",
        "title": "기생충",
        "year": 2019,
        "genre": "드라마",
        "description": "가난한 가족과 부자 가족 사이의 빈부격차를 그린 사회 풍자 드라마"
    },
    {
        "id": "movie4",
        "title": "범죄도시",
        "year": 2017,
        "genre": "범죄",
        "description": "형사가 범죄 조직을 소탕하는 범죄 액션 영화"
    },
    {
        "id": "movie5",
        "title": "범죄도시 2",
        "year": 2022,
        "genre": "범죄",
        "description": "형사와 범죄 조직의 대결을 그린 범죄 액션 영화의 속편"
    },
    {
        "id": "movie6",
        "title": "헤어질 결심",
        "year": 2022,
        "genre": "범죄",
        "description": "산에서 발생한 의문의 죽음(살인 사건)을 수사하던 형사가 피의자에게 이끌리며 벌어지는 미스터리 멜로 영화"
    },
    {
        "id": "movie7",
        "title": "다만 악에서 구하소서",
        "year": 2020,
        "genre": "범죄",
        "description": "청부 살인업자와 범죄 조직의 마지막 거래를 그린 범죄 액션 영화"
    }
]

In [5]:
%pip install langchain langchain_community

  Using cached langchain_community-0.3.24-py3-none-any.whl.metadata (2.5 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.9.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.3.24-py3-none-any.whl (2.5 MB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached marshmallow-3.26.1-py3-none-any.whl (50 kB)
Using cached pydantic_settings-2.9.1-py3-none-any.whl (44 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.1.0-py3-none-any.whl (5.0 kB)

   ---------------------- ----------------- 4/7 [pydantic-settings]
   -----

In [6]:
from langchain.embeddings import OpenAIEmbeddings
import os
# OpenAIEmbeddings 객체 생성 (모델명과 API 키 지정)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 모든 영화 설명에 대해 임베딩 벡터 생성
descriptions = [movie["description"] for movie in movies]        # 설명문 리스트
movie_vectors = embeddings.embed_documents(descriptions)         # 각 설명문에 대한 임베딩 벡터 리스트 생성

# 임베딩 벡터의 차원 확인 (예상: 1536차원)
print(f"임베딩 벡터 차원: {len(movie_vectors[0])}")

C:\Users\USER\AppData\Local\Temp\ipykernel_33032\98617241.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


임베딩 벡터 차원: 1536


In [ ]:
from pinecone import Pinecone,ServerlessSpec

pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

# 인덱스 생성
index_name = "movi2-index"
if not pc.has_index(index_name):
    pc.create_index(name=index_name,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
index = pc.Index(index_name)

In [9]:
# Pinecone에 벡터 업로드 (upsert)
# 각 레코드는 (id, 벡터, metadata)의 형태로 준비
vector_data = []
for movie, vector in zip(movies, movie_vectors):
    # metadata로 title, genre, year, description 저장
    meta = {
        "title": movie["title"],
        "genre": movie["genre"],
        "year": movie["year"],
        "description": movie["description"]
    }
    vector_data.append((movie["id"], vector, meta))

# 벡터들을 Pinecone 인덱스에 업서트
index.upsert(vectors=vector_data)

{'upserted_count': 7}

In [10]:
# 검색 쿼리 예시
query_text = "감성적인 드라마 영화 추천해줘"

# 쿼리 문장을 임베딩 벡터로 변환
query_vector = embeddings.embed_query(query_text)

# Pinecone에서 벡터 유사도 검색 수행 (코사인 유사도 기반)
# 상위 3개의 가장 가까운 벡터를 찾고, 메타데이터를 포함하여 반환
result = index.query(vector=query_vector, top_k=2, include_metadata=True)

# 결과 출력: 각 결과의 제목, 연도, 장르를 표시
for match in result["matches"]:
    info = match["metadata"]
    print(f"{info['title']} - {info['year']} ({info['genre']})")

기생충 - 2019.0 (드라마)
7번방의 선물 - 2013.0 (드라마)


In [11]:
# 메타데이터 기반 검색
# 메타데이터 필터를 활용한 검색: 2020년 이후 개봉한 영화들 중 상위 3개 반환
query_text2 = "영화"  # 매우 일반적인 쿼리
query_vector2 = embeddings.embed_query(query_text2)

# year 필터 적용 (year >= 2020인 항목만 대상)
filter_condition = {"year": {"$gte": 2020}}

result2 = index.query(vector=query_vector2, top_k=3, filter=filter_condition, include_metadata=True)

for match in result2["matches"]:
    info = match["metadata"]
    print(f"{info['title']} - {info['year']} ({info['genre']})")

범죄도시 2 - 2022.0 (범죄)
헤어질 결심 - 2022.0 (범죄)
다만 악에서 구하소서 - 2020.0 (범죄)


In [12]:
# 쿼리 필터 - 하이브리드검색
# 하이브리드 검색 예시: "2020년 이후의 범죄 영화 보여줘"
hybrid_query = "2020년 이후의 범죄 영화 보여줘"

# 쿼리 임베딩 벡터 생성
hybrid_vector = embeddings.embed_query(hybrid_query)

# 필터 조건 설정: 2020년 이후 & 장르 범죄
hybrid_filter = {
    "year": {"$gte": 2020},
    "genre": {"$eq": "범죄"}
}

# Pinecone 검색 실행 (쿼리+필터)
result3 = index.query(vector=hybrid_vector, top_k=3, filter=hybrid_filter, include_metadata=True)

for match in result3["matches"]:
    info = match["metadata"]
    print(f"{info['title']} - {info['year']} ({info['genre']})")


다만 악에서 구하소서 - 2020.0 (범죄)
범죄도시 2 - 2022.0 (범죄)
헤어질 결심 - 2022.0 (범죄)
